In [ ]:
import pandas as pd
import gensim
from gensim.models import Word2Vec

In [ ]:
import nltk
import re

In [ ]:
#nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
data1 = pd.read_csv('winemag-data_first150k.csv')
data2 = pd.read_csv('winemag-data-130k-v2.csv')

In [ ]:
#Panen andmed kokku
dataset = pd.concat([data1,data2])

In [ ]:
#Vaatan, mis toimub
dataset.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery,taster_name,taster_twitter_handle,title
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,NaN,NaN,NaN
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez,NaN,NaN,NaN
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,NaN,NaN,NaN
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,NaN,NaN,NaN
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude,NaN,NaN,NaN


In [ ]:
#Suvaline hulk kirjeldusi (25000)
subset = dataset.description.iloc[:25000]

In [ ]:
#Esimene kirjeldus näiteks
subset[0]

'This tremendous 100% varietal wine hails from Oakville and was aged over three years in oak. Juicy red-cherry fruit and a compelling hint of caramel greet the palate, framed by elegant, fine tannins and a subtle minty tone in the background. Balanced and rewarding from start to finish, it has years ahead of it to develop further nuance. Enjoy 2022–2030.'

In [ ]:
reviews_lower = [x.lower() for x in subset]

In [ ]:
reviews_lower[0]

'this tremendous 100% varietal wine hails from oakville and was aged over three years in oak. juicy red-cherry fruit and a compelling hint of caramel greet the palate, framed by elegant, fine tannins and a subtle minty tone in the background. balanced and rewarding from start to finish, it has years ahead of it to develop further nuance. enjoy 2022–2030.'

In [ ]:
#Puhastan teksti sodist, aga võib vaielda, et äkki ikkagi on vaja mingit tüüpi arve säilitada (nt aastaarvud)
cleaned_reviews = [re.sub('[^a-zA-Z]', ' ', x) for x in reviews_lower]
cleaned_reviews = [re.sub(r'\s+', ' ', x) for x in cleaned_reviews]

In [ ]:
cleaned_reviews[0]

'this tremendous varietal wine hails from oakville and was aged over three years in oak juicy red cherry fruit and a compelling hint of caramel greet the palate framed by elegant fine tannins and a subtle minty tone in the background balanced and rewarding from start to finish it has years ahead of it to develop further nuance enjoy '

In [ ]:
reviews = [nltk.sent_tokenize(x) for x in cleaned_reviews]

In [ ]:
reviews = [y for x in reviews for y in x]

In [ ]:
reviews[0]

'this tremendous varietal wine hails from oakville and was aged over three years in oak juicy red cherry fruit and a compelling hint of caramel greet the palate framed by elegant fine tannins and a subtle minty tone in the background balanced and rewarding from start to finish it has years ahead of it to develop further nuance enjoy'

In [ ]:
words = [nltk.word_tokenize(x) for x in reviews]

In [ ]:
for i in range(len(words)):
    words[i] = [w for w in words[i] if w not in stopwords.words('english')]

In [ ]:
#Põhimõtteliselt on korpus valmis
words[0]

['tremendous',
 'varietal',
 'wine',
 'hails',
 'oakville',
 'aged',
 'three',
 'years',
 'oak',
 'juicy',
 'red',
 'cherry',
 'fruit',
 'compelling',
 'hint',
 'caramel',
 'greet',
 'palate',
 'framed',
 'elegant',
 'fine',
 'tannins',
 'subtle',
 'minty',
 'tone',
 'background',
 'balanced',
 'rewarding',
 'start',
 'finish',
 'years',
 'ahead',
 'develop',
 'nuance',
 'enjoy']

In [ ]:
# Mudel
model = gensim.models.Word2Vec(words, vector_size = 200 , window = 4 , min_count = 15,
                               seed = 1000, hs = 0, negative = 10,workers=16, epochs = 25)

In [ ]:
#Kümme kõige lähedasemat sõna sõnale "magus"
model.wv.most_similar('sweet')

[('syrup', 0.40505334734916687),
 ('upfront', 0.395243763923645),
 ('preserves', 0.38988175988197327),
 ('ripe', 0.38402512669563293),
 ('sweetness', 0.3738795518875122),
 ('exotic', 0.3501429259777069),
 ('filled', 0.34437164664268494),
 ('pie', 0.34066352248191833),
 ('baked', 0.3323270082473755),
 ('butterscotch', 0.3308701515197754)]

In [ ]:
#Kümme kõige lähedasemat sõna sõnale "kuiv"
model.wv.most_similar('dry')

[('dryness', 0.4438883066177368),
 ('semisweet', 0.3660918176174164),
 ('cutting', 0.35722261667251587),
 ('bite', 0.355294406414032),
 ('lip', 0.35496851801872253),
 ('brisk', 0.3342866599559784),
 ('spry', 0.33288702368736267),
 ('brightened', 0.32236072421073914),
 ('moderately', 0.31317660212516785),
 ('drinkable', 0.3089016079902649)]

Päris asjani on tükk maad veel. Ma jooksutasin tervel datal ka (mingi paar tundi läks) ja tulid palju paremad vektorid, aga siis pani arvuti pange :') Mu arust võiks jõuda selleni, et saaks veinisoovitusi nii karakterit kirjeldavate märksõnade kui veininime järgi (ehk mis veinid oleksid konkreetsele veinile kõige sarnasemad). Word2veci saab ära kasutada selles mõttes, et märksõna järgi otsides ei pea istuma me kinni ühe kindla sõna küljes (nt kui otsime kuiva veini, siis me ei saa vastu ainult neid, mille kirjelduses on konkreetselt sõna "kuiv"), vaid saame võtta arvesse ka selle sünonüüme/semantiliselt lähedasi sõnu. Siinse katsetuse põhjal saame ehk algatada arutelu?